### Access respective Azure storage container and blob to fetch data.

In [0]:
# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "censusdatacontainer"
blob_relative_path = "release/us_population_zip/"
blob_sas_token = r""

### Construct remote blob path to access the parquet files remotely.

In [0]:
# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

### Read the parquet files and load the data into Temporary view.

In [0]:
# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

### SQL Operations on the Temporary View.

Display total number of rows

In [0]:
#Display total number of rows:
print("Count of rows :")
display(spark.sql('SELECT count(*) as Count FROM source'))

Display top 10 rows

In [0]:
# Display top 10 rows
display(spark.sql('SELECT * FROM source LIMIT 10'))

Display top 10 rows by specifying column names

In [0]:
display(spark.sql('SELECT year, ZipCode, race, sex, minage, maxage, population FROM source LIMIT 10'))

Display list of unique years from the data

In [0]:

display(spark.sql('SELECT distinct year FROM source'))

Display the list of unique ZipCodes from the data

In [0]:
display(spark.sql('SELECT distinct ZipCode FROM source'))

Display unique race from the data

In [0]:
display(spark.sql('SELECT distinct race FROM source'))

Display total population by year

In [0]:
display(spark.sql('SELECT year, SUM(population) AS Total_Population FROM source group by year'))

Display total population by year and ZipCode

In [0]:
display(spark.sql('SELECT year, ZipCode, SUM(population) AS Total_Population FROM source group by year, ZipCode order by year, ZipCode'))

Display top 100 populated ZipCode by year

In [0]:
display(spark.sql('SELECT year, ZipCode, SUM(population) AS Total_Population FROM source group by year, ZipCode order by Total_Population desc, year, ZipCode LIMIT 100'))

Display total population by year, ZipCode and race

In [0]:
display(spark.sql('SELECT year, ZipCode, race, SUM(population) AS Total_Population FROM source group by year, ZipCode,race order by year, ZipCode, race'))

Display total population by year and sex

In [0]:
display(spark.sql('SELECT year, sex, SUM(population) AS Total_Population FROM source group by year,sex'))

Display most populated ZipCode

In [0]:
display(spark.sql('SELECT ZipCode, SUM(population) AS Total_Population FROM source GROUP BY ZipCode ORDER BY Total_Population DESC LIMIT 1'))

Display most populated ZipCode by year

In [0]:
county_query = "SELECT year, ZipCode, SUM(population) AS Total_Population FROM source GROUP BY year, ZipCode ORDER BY Total_Population DESC LIMIT 1"
display(spark.sql(county_query))